# Applied Natural Language Processing 955G5
## Computer Based Examination, 2024

Remember, you can add cells and change their type (between code and text/markdown) as required to answer the questions.

In [ ]:
# update your candidate number here
candidate_number = 11111111

#Question 2 (50 marks)

This questions is about document classification.

You will construct a model using the sentences in the lists `spam` and `notspam` and apply the model to the sentence assigned to the variable `unlabelled`.

In [1]:
### do not change the code in this cell
# make sure you run this cell

spam =["free laser hair treatment will improve your dating prospects",
       "leverage this weird blockchain hack to generate free cash",
       "new blockchain dating app needs single men",
       "change your life prospects with a single liposuction treatment",
       "one weird trick for free real estate"]

notspam=["can we reschedule for next week",
         "i will be back in the office on monday ",
         "thanks for all your hard work this week",
         "i like your proposal in principle but will need to see more detail",
         "will you be free at 3 pm"]

unlabelled="blockchain proposal"


a)

i) Use the NLTK word tokenizer to tokenize the sentences in the lists `spam` and `notspam` and produce two new lists called `spam_tokenized` and `notspam_tokenized`.

For example, the list `["this is a sentence", "this is another"]` would become `[["this", "is", "a", "sentence"], ["this", "is", "another"]]`.

[5 marks]

In [4]:
import nltk
from nltk import word_tokenize


spam_tokenized = [word_tokenize(s_sent) for s_sent in spam]
notspam_tokenized = [word_tokenize(ns_sent) for ns_sent in notspam]

In [8]:
spam_tokenized[0:2]

[['free',
  'laser',
  'hair',
  'treatment',
  'will',
  'improve',
  'your',
  'dating',
  'prospects'],
 ['leverage',
  'this',
  'weird',
  'blockchain',
  'hack',
  'to',
  'generate',
  'free',
  'cash']]

ii) Construct a training dataset from the lists of spam and not spam tokenized sentences, consisting of a list of two element tuples in which the first element contains the list of tokens and the second element is either `1` indicating spam or `0` indicating not spam.

For example, the spam sentence `["free", "crypto"]` would be represented in the training set as `(["free", "crypto"], 1)`

[4 marks]

In [10]:
spam_labeled = [(sentence, 1) for sentence in spam_tokenized]
notspam_labeled = [(sentence, 0) for sentence in notspam_tokenized]
training = spam_labeled + notspam_labeled
print(training[:2])

[(['free', 'laser', 'hair', 'treatment', 'will', 'improve', 'your', 'dating', 'prospects'], 1), (['leverage', 'this', 'weird', 'blockchain', 'hack', 'to', 'generate', 'free', 'cash'], 1)]


iii) Convert the sentences in the training dataset to a bag-of-words representation using the FreqDist class. For example, the document `["this", "is", "a", "sentence"]` would become `FreqDist({'this': 1, 'is': 1, 'a': 1, 'sentence': 1})`. The result should a be a list of pairs, in which one item is the bag-of-words sentence representation and the other is the `0`/`1` label.

[6 marks]

In [13]:

from nltk import FreqDist
BoW = [[FreqDist(sent),label] for sent, label in training]
BoW[0]

[FreqDist({'free': 1, 'laser': 1, 'hair': 1, 'treatment': 1, 'will': 1, 'improve': 1, 'your': 1, 'dating': 1, 'prospects': 1}),
 1]

b)

i) From the training data, calculate the prior probabilities, $p(c)$, of a sentence being in each class, $c \in \{ 0 , 1 \}$.
$$ p(c) = \frac{freq(c)}{N}$$
Here, $freq(c)$ is the number of sentences with label $c$, and $N$ is the total number of documents.

The result should be a list, called `class_priors`, with two entries corresponding to the two classes. The first item in the list, `class_priors[0]`, should correspond to the probability of a sentence not being spam and the last item, `class_priors[1]`, should correspond to the probability of a sentence being spam.  

So, for example, if 90% of the sentences were not spam, then `class_priors` would take the value `[0.9, 0.1]`.

[6 marks]

In [14]:
total = len(training)
counter = 0
for s,label in training:
    counter += label

class_priors = [
    counter/total,
    (total-counter)/total,
]

class_priors

[0.5, 0.5]

ii) Calculate the conditional probabilities, $p(f|c)$, of each token feature, $f$, given each sentiment class, $c$.
$$ p(f|c) = \frac{freq(f,c)}{freq(c)}$$

Where $freq(f,c)$ is the number of documents in class $c$ containing feature $f$ and $freq(c)$ is the total number of documents in class $c$.

The result should be a dictionary of lists called `cond_probs`, with the keys of the outer dictionary being token features, and the corresponding values being a list of the two conditional class probabilities.

For example, the token `'to'` occurs once in the five spam sentences and once in the five non-spam sentences. So, `cond_probs['to']` should contain the value `[0.2,0.2]`.

[10 marks]

In [127]:
doc_spam_count = 0

cond_probs = {}

for sent,label in training:
    doc_spam_count += label
    for word in set(sent):
        word_lst = cond_probs.get(word,[0,0])
        word_lst[label] += 1
        cond_probs[word] = word_lst

print(cond_probs["to"])

doc_not_spam_count = len(training) - doc_spam_count

for key in cond_probs.keys():
    current = cond_probs[key] # direct reference
    current[0] = 0 if current[0] == 0 else current[0]/doc_not_spam_count
    current[1] = 0 if current[1] == 0 else current[1]/doc_spam_count

print(cond_probs["to"])

[1, 1]
[0.2, 0.2]


iii) For each token in the sentence `unlabelled`, print out the conditional probabilities for that token. Calculate and print the final Naive Bayes probabilities of the two classes for the whole sentence.

[6 marks]

In [130]:
for token in word_tokenize(unlabelled):
    print(f"Conditonal Probabilities for '{token}' is {cond_probs[token]}")

for class_index in range(len(class_priors)):
    calcuate = class_priors[class_index]
    for token in word_tokenize(unlabelled):
        calcuate = 0 if cond_probs[token][class_index]==0 else calcuate * cond_probs[token][class_index]
    print(f"Naive Nayes Probability for Class '{class_index}' is {calcuate}")

Conditonal Probabilities for 'blockchain' is [0, 0.4]
Conditonal Probabilities for 'proposal' is [0.2, 0]
Naive Nayes Probability for Class '0' is 0.0
Naive Nayes Probability for Class '1' is 0


c)

i) Explain the ideas behind the Naive Bayes model.

Starting from the assumption that we want to find the class that maximises  $p(class|document)$ , explain how Bayes theorem is used and what naive assumption is made about the features in the document. Describe the priors and conditional probabilities that are used to predict the most likely class for a document.

[5 marks]

In order to understand the idea behind the Naive Bayes model we first need to instantiate our problem. 

We have any number data instances which are documents. A document is a data structure which is made up of a sequence features with a given order (words, punctuation, numbers). We can respresent this an tuple of features. We use a tuple because the features are indexed in a given order so it retains the format of a sentence or paragraph:

$$ f_1^2 = (f_1, f_2,...,f_n) $$

We want to assign our documents a class, this is something that they belong, i.e. categorizing articles in a newspaper as "football", "politics" etc. With this, there exists a possible set of classes $C$. An individual class from within the set is denoted $c$ which can be denoted $c \in C$. 

The goal of a "model" is to look at the features $f_1^2$ and assign a class based on the features. If the number of classes $C > 1$ then the selection of a class with be derived based on the class which is most likely, this is written $P(c|f_1^2)$

Bayes Theorem provides us with a framework to executre this allocation. 

$$\operatorname*{argmax}_{c} P(c|f_1^n) = \operatorname*{argmax}_{c} \frac{P(f_1^n|c) \cdot P(c)}{P(f_1^n)}$$

This equation is made up of 5 parts: 
- Prior
- Conditional Probability/Likelihood
- Evidence 
- Posterior Probability
- Argmax

The Prior $P(c)$ is the distribution of the class accross all instances. If there all 100 pieces of data, 50 are labelled X and 50 labelled Y then `Class X = 0.5` and `Class Y = 0.5`. In the structure of Bayes, it tells of the before information. Before we know anything about a peice of data, what is the global assignments of classes. 

The Conditional Probability/Likelihood $P(f_1^n|c)$ says, given we know the class of something, what is the max probability that is comes from some set of features. In Bayes it represents the update mechanism. Now that we have an actual data instance, we use it to improve and inform the prior distribution. 

The Evidence $P(f_1^2)$ is probability that a data instances in the set of all possible feature sets. It should be notes that this probability is the same for all feature sets, therefore, for Bayes, this is just a normalization term. It ensures that all final probabilties sum to one. However, in terms of ratios, it doesn't do anything. The highest result will still be the most likely even if the number computed isn't a real probability. As a result this term is often dropped for simplicity. 

On the left handside of the equation is the posterior $P(c|f_1^n)$. This is outcome of updating the prior with the conditional probability. It's tells us, given we have seen a particular feature set, what class is it most likely from. 

Finally, there is argmax. This tells from all of the possible feature sets, select the one that is most likely. 

[naive assumptions to simplify, avoid huge computations, curse of dims, tiny miniscule probs, sentence is a sparse vector of all possible sentences, likely 0 exact occur, joint probabilities that account for the specific sequence and context of every word, transfers data structure to bow instead of list/tuple, product of features]

In its "pure" form, calculating the joint probability $P(f_1, f_2, ..., f_n|c)$ is computationally intractable due to the "curse of dimensionality." To calculate the exact sequence probability, we would need to account for the context and dependencies of every word (the Chain Rule). In most datasets, specific long sequences are unique, leading to sparse vectors where the probability of an exact match is likely zero.

To solve this, we make the Naive Independence Assumption: we assume that every feature $f_i$ is conditionally independent of every other feature given the class. This simplifies the likelihood into a product of individual probabilities:

$$P(f_1, ..., f_n|c) \approx P(f_1|c) \times P(f_2|c) \times ... \times P(f_n|c)$$

This transforms the data structure from a specific ordered sequence into a Bag-of-Words representation. While we lose word order and context, we gain a model that can effectively estimate probabilities from smaller training sets by looking at individual word frequencies.

ii) Should we apply smoothing to the data above? Justify your answer.

[3 marks]

Most NLP applications struggle with the concept of sparseness. There is a huge number of features (words) in a langauge to think of and outside the common connective words, most words by definition are use relatively sparsely in general. The specific issue we faced in our example was that the words in the test phrase "blockchain proposal" did not occur in both classes. Taking "blockchain" as an example, it occured in the `spam` class meaning the NB model computed a probability of 0.4 for it, however, it did not occur in the `not_spam` class meaning the probability here had to be 0. When computing the probability for a whole sentence, whether we are applying the naive assumption or not, we need to in some way take the product of all features in a sentence. This means a 0 probability feature will cause the probability for the sentence to be computed as 0. We can correct for this by using a smooth technique. Here, we adjust our computations so that by default, all words seen in the entire vocabularly, in this case both spam and not_spam words, are appended with an initalising count of 1. That means the `spam` blockchain count would move from 2 to 3 and the `not_spam` count would move from 0 to 1. This means that we go to compute the class posterior probabilties again, it will not some out as 0 as we will have an eligible result which can be used. 

iii) Recalculate the conditional probabilities after applying add one smoothing.

[3 marks]

In [129]:
_doc_spam_count = 0

sm_cond_probs = {}

for _sent,_label in training:
    _doc_spam_count += _label
    for _word in set(_sent):
        _word_lst = sm_cond_probs.get(_word,[0,0])
        _word_lst[_label] += 1
        sm_cond_probs[_word] = _word_lst

print(sm_cond_probs["to"])

_doc_not_spam_count = len(training) - _doc_spam_count

for _key in sm_cond_probs.keys():
    _current = sm_cond_probs[_key] # direct reference
    _current[0] = (_current[0]+1)/(_doc_not_spam_count + 2)
    _current[1] = (_current[1]+1)/(_doc_spam_count + 2)

print(sm_cond_probs["to"])

[1, 1]
[0.2857142857142857, 0.2857142857142857]


iv) Make a Naive Bayes prediction for the `unlabelled` sentence using the smoothed probabilities.

[2 Marks]

In [131]:
for token in word_tokenize(unlabelled):
    print(f"Conditonal Probabilities for '{token}' is {sm_cond_probs[token]}")

for class_index in range(len(class_priors)):
    calcuate = class_priors[class_index]
    for token in word_tokenize(unlabelled):
        calcuate = 0 if sm_cond_probs[token][class_index]==0 else calcuate * sm_cond_probs[token][class_index]
    print(f"Naive Nayes Probability for Class '{class_index}' is {calcuate}")


Conditonal Probabilities for 'blockchain' is [0.14285714285714285, 0.42857142857142855]
Conditonal Probabilities for 'proposal' is [0.2857142857142857, 0.14285714285714285]
Naive Nayes Probability for Class '0' is 0.02040816326530612
Naive Nayes Probability for Class '1' is 0.03061224489795918
